In [46]:
# first question
import psycopg2

#establishing the connection
conn = psycopg2.connect(
   database='dvdrental', user='postgres', password='1', host='127.0.0.1', port= '5432'
)
cursor = conn.cursor()

sql ='''
    
    CREATE or replace FUNCTION public.get_number(IN phone_number character(11))
    returns varchar(100)
    language 'plpgsql'
    as $BODY$
    begin
    if left(phone_number, 2) = '09' then
        return 'mobile phone number';
    elseif left(phone_number, 3) = '021' then
        return concat('city code: ', left(phone_number, 3), ', last 8 digits: ',
                                    substring(phone_number from 4 for 8), ', city: Tehran');
    elseif left(phone_number, 3) = '031' then
        return concat('city code: ', left(phone_number, 3), ', last 8 digits: ',
                                    substring(phone_number from 4 for 8), ', city: Esfahan');
    elseif left(phone_number, 3) = '077' then
        return concat('city code: ', left(phone_number, 3), ', last 8 digits: ', 
                                    substring(phone_number from 4 for 8), ', city: Bushehr');
    else 
        return 'Invalid phone number';
    end if;
    end;
    $BODY$;
    
'''
cursor.execute(sql)
conn.commit()

conn.close()

In [110]:
# second question
import psycopg2

#establishing the connection
conn = psycopg2.connect(
   database='dvdrental', user='postgres', password='1', host='127.0.0.1', port= '5432'
)
cursor = conn.cursor()

sql ='''
    DROP FUNCTION films_between_two_dates(date,date);
    CREATE or replace FUNCTION public.films_between_two_dates(IN date1 date, IN date2 date)
    returns table(film_title varchar(100), rental_id integer, customer_id smallint)
   
    as $BODY$
    begin
        return query select f.title, r.rental_id, r.customer_id from rental r 
        inner join inventory i on i.inventory_id=r.inventory_id 
        inner join film f on f.film_id=i.film_id
        where rental_date between date1 and date2;
    end;
    $BODY$
     language 'plpgsql';
    
'''
cursor.execute(sql)
conn.commit()

conn.close()

In [146]:
# third question
import psycopg2

#establishing the connection
conn = psycopg2.connect(
   database='dvdrental', user='postgres', password='1', host='127.0.0.1', port= '5432'
)
cursor = conn.cursor()

sql ='''
    
    CREATE or replace FUNCTION public.customers_did_not_return(IN date1 date, IN date2 date)
    returns table(rental_id integer, first_name varchar(100), last_name varchar(100), rental_date timestamp
    ,email varchar(100), address_id smallint)
   
    as $BODY$
    begin
        return query select r.rental_id, c.first_name, c.last_name, r.rental_date
        , c.email, c.address_id
        from
        rental r  
        left join customer c on r.customer_id=c.customer_id
        where r.return_date is null and 
        exists(select f.rental_id from public.films_between_two_dates(date1, date2) f where f.rental_id=r.rental_id);
    end;
    $BODY$
     language 'plpgsql';
    
'''
cursor.execute(sql)
conn.commit()

conn.close()

In [193]:
# forth question
import psycopg2

#establishing the connection
conn = psycopg2.connect(
   database='dvdrental', user='postgres', password='1', host='127.0.0.1', port= '5432'
)
cursor = conn.cursor()

sql ='''
    CREATE OR REPLACE FUNCTION public.check_valid_rental()
        RETURNS TRIGGER
        language 'plpgsql'
    as $BODY$
    BEGIN
        
        IF NEW.return_date::DATE - NEW.rental_date::DATE > (select f.rental_duration 
        from film f right join inventory i on i.film_id=f.film_id
        where NEW.inventory_id = i.inventory_id) then
            INSERT INTO public.log values(
                    NEW.customer_id
                    ,EXTRACT(epoch FROM (NEW.return_date::DATE - NEW.rental_date))/86400);
        
        end if;
         
        RETURN NEW;
    END;
    $BODY$;
    
    CREATE or replace TRIGGER check_return_data
      AFTER INSERT OR UPDATE
      ON rental
      FOR EACH ROW
      EXECUTE PROCEDURE public.check_valid_rental();
   
    
    
    
'''
cursor.execute(sql)
conn.commit()

conn.close()

In [212]:
# fifth question
import psycopg2

#establishing the connection
conn = psycopg2.connect(
   database='dvdrental', user='postgres', password='1', host='127.0.0.1', port= '5432'
)
cursor = conn.cursor()

sql ='''
     CREATE OR REPLACE PROCEDURE increase_rent(
         amount int,
         value2 int
     ) LANGUAGE plpgsql as $BODY$ 
     begin
     
     update film set rental_duration = rental_duration + amount;
     
     end;
     $BODY$
     
    
    
'''
cursor.execute(sql)
conn.commit()

conn.close()

In [222]:
# test functions
import psycopg2

#establishing the connection
conn = psycopg2.connect(
   database='dvdrental', user='postgres', password='1', host='127.0.0.1', port= '5432'
)
cursor = conn.cursor()

sql ='''
    --select public.get_number('09124567895')
    --select public.films_between_two_dates('2005-3-10', '2007-4-10');
    --select public.customers_did_not_return('2005-3-10', '2007-4-10');
    --INSERT INTO rental values(16049, '2005-8-23 22:50:12', 2666, 393, '2005-9-30 1:1:12', 2, '2006-2-16 2:30:53');
    --UPDATE rental set return_date='2005-9-15 1:10:00' where rental_id=16048;
    --select * from log;
    call public.increase_rent(1,25);
    select  film_id, rental_duration from film order by film_id;
'''
cursor.execute(sql)
conn.commit()
result = cursor.fetchall()
for i in result:
    print(i)

conn.close()

(1, 10)
(2, 7)
(3, 11)
(4, 9)
(5, 10)
(6, 7)
(7, 10)
(8, 10)
(9, 7)
(10, 10)
(11, 10)
(12, 10)
(13, 8)
(14, 10)
(15, 9)
(16, 10)
(17, 7)
(18, 10)
(19, 10)
(20, 8)
(21, 7)
(22, 10)
(23, 7)
(24, 10)
(25, 7)
(26, 7)
(27, 11)
(28, 9)
(29, 9)
(30, 8)
(31, 9)
(32, 10)
(33, 9)
(34, 10)
(35, 8)
(36, 11)
(37, 7)
(38, 10)
(39, 9)
(40, 8)
(41, 8)
(42, 9)
(43, 10)
(44, 9)
(45, 9)
(46, 7)
(47, 9)
(48, 7)
(49, 10)
(50, 7)
(51, 9)
(52, 10)
(53, 9)
(54, 9)
(55, 10)
(56, 10)
(57, 8)
(58, 10)
(59, 8)
(60, 7)
(61, 9)
(62, 9)
(63, 10)
(64, 10)
(65, 7)
(66, 10)
(67, 9)
(68, 9)
(69, 7)
(70, 11)
(71, 7)
(72, 10)
(73, 9)
(74, 8)
(75, 10)
(76, 8)
(77, 9)
(78, 11)
(79, 9)
(80, 11)
(81, 10)
(82, 7)
(83, 9)
(84, 11)
(85, 8)
(86, 10)
(87, 11)
(88, 11)
(89, 11)
(90, 7)
(91, 9)
(92, 11)
(93, 8)
(94, 11)
(95, 9)
(96, 8)
(97, 11)
(98, 8)
(99, 11)
(100, 11)
(101, 7)
(102, 8)
(103, 11)
(104, 8)
(105, 10)
(106, 8)
(107, 8)
(108, 10)
(109, 7)
(110, 8)
(111, 7)
(112, 8)
(113, 8)
(114, 7)
(115, 9)
(116, 8)
(117, 10)
(118, 1